## ROC extraction

on commence par regarder les documents ou l on observe le mot center


In [ ]:
# SYSTEM
from os import listdir
import sys
sys.path.insert(0,'/Users/greghor/Documents/taf/UGent/python_prog/py_commonFunc/')
from PyCommonFun import *

#NLTK
import nltk, re, pprint
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import words
from nltk.tokenize import RegexpTokenizer
from nltk.metrics import *
from nltk.text import Text

#GENERAL
import numpy as np
import pandas as pd
import pickle
import string

In [ ]:
# data path
curDir=%pwd
if curDir=='/Users/greghor/Documents/taf/dataScience/Euroclear_hackathon/notebook':
    dataPath='/Users/greghor/Documents/taf/dataScience/Euroclear_hackathon/dataset/data_0_labels' 
                   
else:
    dataPath='../../data_0'

In [ ]:
curDir

In [ ]:
dataPath


In [ ]:
cities['c']

In [ ]:
# get the solution
labels=pd.read_csv(dataPath + '/labels.csv')
ROCTmp=pd.read_csv('ROC_fullList.csv',delimiter=";")
cities_groundTruth=pd.read_csv(dataPath + '/cities.csv')
ROC_list=['target 1','target 2','target'] # add target 1 and target 2
# lowerise ROC and remove ponctuation
for roc in ROCTmp['CIT_NM'].values:
    roc=str(roc).lower().translate(None, string.punctuation)
    ROC_list.append(roc.rstrip())

In [ ]:
np.unique(cities_groundTruth['City.Name'].values)

In [ ]:
def getContext(s,pattern,context=list(),width=100):
    
    
    loc=s.find(pattern)
    
    if loc == -1:
        return context
    elif loc-width < 0:
        context.append(s[:loc+width])
        return getContext(s[loc+len(pattern):],pattern,context,width)
    elif loc + width > len(s):
        context.append(s[loc-width:])
        return getContext(s[loc+len(pattern):],pattern,context,width)
    else:
        context.append(s[loc-width:loc+width])
        return getContext(s[loc+len(pattern):],pattern,context,width)
    

In [ ]:
s="je vais à la mer, il fait beau, je chante et je danse"
getContext(s,"je",[],width=4)

In [ ]:
len(cities)


In [ ]:
s[0:3]

In [ ]:
gf=0


context_bdc=dict()


for f in labels.fileId:
    fileOpen=open(dataPath + '/txt/' + f + '_ocr.txt')
    text=fileOpen.read()
    text=unicode(text, 'ascii', 'ignore')
    text=str(text).lower()
    # remove some annoying punctutations
    text=text.translate(None, string.punctuation)
    text=text.translate(None,'\r\n\x0c')
   
    
    context_bdc[f]=dict()
    sentence=sent_tokenize(text)
    
    pattern=["Business Day Centre","Financial Centre","Currency Business Day",'Business Day']
    
    for pat in pattern:
        context_bdc[f][pat]=getContext(text,pat.lower(),[],250)
        # il faut impérativement passer, un troisieme argument vide pour que le code 
        # fonctionne mais je ne comprends pas bien pourquoi
            
            
            
    gf+=1
       


In [ ]:
context_bdc['0900045c83aef46e']

In [ ]:
# try to extract the ROC 

extractedROC_dict=dict()

for fid in context_bdc.keys():
    
    extractedROC_dict[fid]=dict()
    
    for pat in context_bdc[fid].keys():
    
        extractedROC_dict[fid][pat]=''
        
        for roc in ROC_list:
            
            if roc in ''.join(context_bdc[fid][pat]):
                
                # not clean but makes things much easier for 
                if roc=="sao paulo":
                    roc="sao-paulo"
                elif roc=='target 1' or roc =='target 2' or roc == 'target':
                    roc='target holidays'
            
                    
                
                if len(extractedROC_dict[fid][pat])==0:
                    extractedROC_dict[fid][pat]=roc
                else:
                    extractedROC_dict[fid][pat]=extractedROC_dict[fid][pat]+'|'+roc
                
                
                
        
        
                 

In [ ]:
extractedROC_df=pd.DataFrame()
extractedROC_df['fileId']=[]
for pat in extractedROC_dict['0900045c83afbad4'].keys():
    extractedROC_df[pat]=[]




In [ ]:
for fid in extractedROC_dict.keys():
    newRow=[fid]
    for pat in extractedROC_dict[fid].keys():
        newRow.append(extractedROC_dict[fid][pat].upper())
    extractedROC_df.loc[len(extractedROC_df)]=newRow

In [ ]:
extractedROC_df

In [37]:
# merge with citiesGreg to check how many results are OK

citiesGreg=pd.read_csv('citiesGreg.csv')
citiesGreg.index=citiesGreg['fileId']
joinedCities=pd.merge(citiesGreg,extractedROC_df,on='fileId')

In [38]:
joinedCities

,Unnamed: 0,isin,citiesName,fileId,Business Day,Financial Centre,Currency Business Day,Business Day Centre
0,0,XS1240015799,NEW YORK,0900045c83a87e4a,NEW YORK,NEW YORK,,
1,1,XS1293137904,TARGET HOLIDAYS,0900045c83c101a8,TARGET,,,TARGET
2,2,XS1313796846,LONDON|TOKYO,0900045c83ac888d,,LONDON|TOKYO,,
3,3,XS1313802677,NEW YORK|TOKYO,0900045c83b42aaf,,NEW YORK|TOKYO,,
4,4,XS1313800895,LONDON|TOKYO,0900045c83b36448,TOKYO,LONDON|TOKYO,,
5,5,XS1267246327,NEW YORK|TARGET HOLIDAYS,0900045c83a715b0,TARGET|NEW YORK,TARGET|NEW YORK,,
6,6,XS1275964507,LONDON|NEW YORK|SAO-PAULO|TOKYO,0900045c8365c27b,LONDON|NEW YORK|SAO-PAULO|TOKYO,LONDON|NEW YORK|SAO-PAULO|TOKYO,,
7,7,XS1329979261,TOKYO,0900045c83aab72e,,TOKYO,,
8,8,XS1312160507,NEW YORK,0900045c83af0dc8,,NEW YORK,,
9,9,XS1253932765,NEW YORK|SINGAPORE,0900045c833b5bd8,BRUSSELS|EUROCLEAR|LUXEMBOURG|NEW YORK|SINGAPORE,NEW YORK|SINGAPORE,,


In [39]:
# check the compliance between ground truth and my solution
for pat in pattern:
    indCheck=joinedCities["citiesName"] == joinedCities[pat]
    joinedCities["check_" + pat ]=indCheck

# reorder the columns for clarity
col=['fileId',
 'isin',
 'citiesName',
 'Financial Centre','check_Financial Centre',
 'Business Day Centre', 'check_Business Day Centre',
 'Currency Business Day', 'check_Currency Business Day',
 'Business Day', 'check_Business Day']

In [40]:
check_extractedCities=joinedCities[col]
check_extractedCities.to_csv('check_extractedCities.csv')

In [ ]:
" election is given to the Noteholders in".index('to the')

In [ ]:
tok=word_tokenize(str(context_bdc["0900045c83a966e3"]))
toto=nltk.Text(tok)

In [ ]:
toto.common_contexts(['Issuer'])

In [ ]:
'Hong Kong' in str(context_bdc["0900045c83a966e3"])

In [ ]:
context_bdc["0900045c83a96696"][471-50:471+50]

In [ ]:
np.where(text=='Issuer')

In [ ]:
toto=re.search('Business Day Centre',text)

In [ ]:
toto.group()

In [ ]:
toto=text.common_contexts("Centre")

In [ ]:
text.findall("<.*><Day Centre><.*>")

In [ ]:
moby = Text(nltk.corpus.gutenberg.words('melville-moby_dick.txt'))

In [ ]:
moby

In [ ]:
moby.common_contexts('whale')

In [ ]:
toto

In [ ]:
toto

In [ ]:
text.concordance('prospectus')


In [ ]:
type(text)

In [ ]:
f

In [ ]:
f

In [ ]:
f

In [ ]:
for f in labels[0

In [ ]:
labels.fileId.values[0]

In [ ]:
for f in labels.fileId:
    print f